In [ ]:
!python -m pip install transformers


In [ ]:
import random
import numpy as np
import torch
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


**Load Datasets:**

In [ ]:
dataset = load_dataset("imdb")

train_texts = dataset["train"]["text"]
train_labels = dataset["train"]["label"]

test_texts = dataset["test"]["text"]
test_labels = dataset["test"]["label"]


**BALANCE THE DATA:**

In [ ]:
def balance_dataset(texts, labels, samples_per_class):
    pos = [(t,l) for t,l in zip(texts, labels) if l == 1]
    neg = [(t,l) for t,l in zip(texts, labels) if l == 0]

    random.shuffle(pos)
    random.shuffle(neg)

    pos = pos[:samples_per_class]
    neg = neg[:samples_per_class]

    data = pos + neg
    random.shuffle(data)

    texts, labels = zip(*data)
    return list(texts), list(labels)

# Balanced & CPU-safe
train_texts, train_labels = balance_dataset(train_texts, train_labels, 600)
test_texts, test_labels = balance_dataset(test_texts, test_labels, 300)

print(len(train_texts), sum(train_labels))


**Tokenization:**

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

train_encodings = tokenizer(
    list(train_texts),
    truncation=True,
    padding=True,
    max_length=128
)

test_encodings = tokenizer(
    list(test_texts),
    truncation=True,
    padding=True,
    max_length=128
)


**Dataset Class:**

In [ ]:
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)


**Load BERT Model**

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2
)


 **Training Arguments:**

In [ ]:
training_args = TrainingArguments(
    output_dir="./bert_cpu",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    save_strategy="no",
    report_to="none"
)


**Metrics Function:**

In [ ]:
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    return {
        "accuracy": accuracy_score(labels, preds),
        "precision": precision_score(labels, preds, zero_division=0),
        "recall": recall_score(labels, preds, zero_division=0),
        "f1": f1_score(labels, preds, zero_division=0)
    }


**Trainer:**

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)


**Train Model:**

In [ ]:
trainer.train()


**Final Evaluation:**

In [ ]:
results = trainer.evaluate()

results


**BERT :Final Evaluation Metrics.**

In [ ]:
import pandas as pd

comparison_df = pd.DataFrame({

    "Accuracy": [results["eval_accuracy"]],
    "Precision": [results["eval_precision"]],
    "Recall": [results["eval_recall"]],
    "F1-Score": [results["eval_f1"]]
})

comparison_df
